<a href="https://colab.research.google.com/github/jfpIE16/SL_I_Tareas/blob/master/ConclusionesyEnsembleLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ensemble learning
José Fernando Pérez Pérez

*josefernando.perez@galileo.edu*


---

En este notebook utilizaremos la técnica Ensemble learning para poder realizar una predicción final sobre el conjunto de pruebas.

In [23]:
import numpy as np
from sklearn.externals import joblib
from scipy import stats
import sklearn.metrics as mt
import pandas as pd

In [7]:
test_data = np.load("/content/drive/My Drive/test_data.npz")
x_test = test_data["x_test"]
y_test = test_data["y_test"]

In [16]:
def get_metrics(y_hat, y):
  '''
  get_metrics: obtiene un conjunto de metricas a partir del conjunto de datos
  real y las predicciones hechas
  Input:
    y_hat - vector de predicciones
    y     - vector de valores reales
  Output:
    metrics:
      [0] - Accuracy
      [1] - F1 Score
      [2] - Precision Score
      [3] - Recall Score
  '''
  metrics = [mt.accuracy_score(y, y_hat, normalize=True),
             mt.f1_score(y, y_hat),
             mt.precision_score(y, y_hat),
             mt.recall_score(y, y_hat, average='weighted')]
  return metrics

In [9]:
# Funcion para realizar inferencias de Regresion Logistica
def inference_log_reg(x, weights):
  def sigmoid(l):
    return (1 / (1 + np.exp(-l)))
  x_f = np.insert(x, 0, 1, axis=1)
  logits = np.matmul(x_f, weights)
  y_hat = 1.0*(sigmoid(logits) > 0.5)

  return y_hat

Definimos una función que se encargara de realizar las predicciones con base en los 3 modelos creados, se cargaran los modelos a partir de los archivos generados por cada modelo.

In [22]:
def titanicPrediction(x):
  # Carga de modelos
  # Cargar el modelo SVM
  svm_model = joblib.load("/content/drive/My Drive/svm_final_3.pkl")
  # Cargamos los pesos para Regresion logistica
  reg_log = np.load("/content/drive/My Drive/reg_log_weights.npz")
  weights = reg_log["W"]
  # Carga modelo Naive Bayes implementado con Scikit-learn
  NB_model = joblib.load("/content/drive/My Drive/final_NB.pkl")

  # Predicciones
  y_hat_svm = svm_model.predict(x)
  y_hat_reglog = inference_log_reg(x, weights)
  y_hat_nv = NB_model.predict(x[:, [2,9]]) # Utilizamos la combinacion optima

  # Creamos una matriz con cada una de las predicciones
  y_hat_matrix = np.column_stack((y_hat_svm, y_hat_reglog, y_hat_nv))

  y_hat_final, _ = stats.mode(y_hat_matrix, axis = 1)

  return y_hat_final, y_hat_matrix

## Predicción final
El valor final es la moda de las 3 columnas, nos indica el valor que se repite más y dicho valor se toma como la predicción final.

In [26]:
y_hat, y_hat_matrix = titanicPrediction(x_test)

resultados = pd.DataFrame(y_hat_matrix,
                           columns = ["SVM", "Logistic_Regression", "Naive_Bayes"])
resultados

,SVM,Logistic_Regression,Naive_Bayes
0,0.0,1.0,1.0
1,0.0,0.0,0.0
2,1.0,1.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
...,...,...,...
174,0.0,0.0,0.0
175,0.0,0.0,0.0
176,1.0,1.0,0.0
177,0.0,0.0,0.0


# Accuracy final

In [21]:
print("Accuracy : %s"%round(get_metrics(y_hat, y_test)[0],2))

Accuracy : 0.84


Se obtuvo una exactitud del **84%** sobre el conjunto de datos de prueba, cabe destacar que no se habían utilizado estos datos en ningún momento de la implementación del proyecto.

# Pasos a seguir y K-folds

- Investigar y practicar seguido el tema de *feature engineering*, dado que es un tema muy importante en la implementación de modelos predictivos.
- La técnica **K-folds** permite tomar un subconjunto aleatorio de datos a partir del conjunto original, lo cual sería una ventaja para evitar valores de *sesgo* altos y nos permite tener mejores modelos por la aleatoriedad de los subconjuntos.


# Conclusiones

- Este proyecto me ha ayudado mucho personalmente en el sentido que he perdido un poco el miedo al momento de analizar los datos y con el complemento de las clases se que esta pasando en cada parte del proyecto. Los modelos dejaron de ser cajas negras, ahora ya entiendo un poco mejor su funcionamiento.
- En mi caso la mayor dificultad fue el ajuste de los modelos, principalmente en *Logistic Regression*.

## Lecciones aprendidas

- *Feature engineering* es sumamente necesario para un científico de datos, tengo que aprender muchas tecnicas tanto de numpy, pandas y programación convencional para poder llevar a cabo este proceso de una forma mas fluida.
- Las lecturas me han ayudado bastante a entender los modelos y como poder mejorar los parametros, sin embargo, creo que aún así debo leer más y complementar con prácticas.
- De mi parte tengo que dar una mayor ventana de tiempo para la implementación del proyecto y no dejarlo para las ultimas semanas. En el curso siguiente tratare de avanzar mediante el flujo normal de la clase.

## Recomendaciones

- Para *Logistic Regression* me quedo pendiente utilizar *L1 Regularization* y consideró que habría sido de mucha utilidad ya que creo que mi conjunto de datos finales tenía algunas columnas que no aportaban nada al modelo.

# Deployment
Se utiliza la función titanicPrediction para realizar inferencia de conjuntos de datos particulares.

In [44]:
random_data = np.array(([1,12.,0,1,1,1,0,1,0,85],
                       [0,6.,1,0,0,1,0,1,0,14]))
y_hat, y_hat_mat = titanicPrediction(random_data)
y_hat_mat

array([[0., 0., 1.],
       [0., 0., 0.]])